#### This notebook demonstrates the use of the Reject Option Classification (ROC) post-processing algorithm for bias mitigation for Adult race.
- The debiasing function used is implemented in the `RejectOptionClassification` class.
- Divide the dataset into training, validation, and testing partitions.
- Train classifier on original training data.
- Estimate the optimal classification threshold, that maximizes balanced accuracy without fairness constraints.
- Estimate the optimal classification threshold, and the critical region boundary (ROC margin) using a validation set for the desired constraint on fairness. The best parameters are those that maximize the classification threshold while satisfying the fairness constraints.
- The constraints can be used on the following fairness measures:
    * Statistical parity difference on the predictions of the classifier
    * Average odds difference for the classifier
    * Equal opportunity difference for the classifier
- Determine the prediction scores for testing data. Using the estimated optimal classification threshold, compute accuracy and fairness metrics.
- Using the determined optimal classification threshold and the ROC margin, adjust the predictions. Report accuracy and fairness metric on the new predictions.

In [1]:
%matplotlib inline
# Load all necessary packages
import sys
sys.path.append("../")
import numpy as np
from tqdm import tqdm
from warnings import warn

from aif360.datasets import BinaryLabelDataset
from aif360.datasets import StandardDataset
from aif360.metrics import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.postprocessing.reject_option_classification\
        import RejectOptionClassification
from common_utils import compute_metrics

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from ipywidgets import interactive, FloatSlider
import pandas as pd
import pickle

`load_boston` has been removed from scikit-learn since version 1.2.

The Boston housing prices dataset has an ethical problem: as
investigated in [1], the authors of this dataset engineered a
non-invertible variable "B" assuming that racial self-segregation had a
positive impact on house prices [2]. Furthermore the goal of the
research that led to the creation of this dataset was to study the
impact of air quality but it did not give adequate demonstration of the
validity of this assumption.

The scikit-learn maintainers therefore strongly discourage the use of
this dataset unless the purpose of the code is to study and educate
about ethical issues in data science and machine learning.

In this special case, you can fetch the dataset from the original
source::

    import pandas as pd
    import numpy as np

    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df

#### Load dataset and specify options


Huangrui's ADULT PREPROCESS

Only modify the sex and race to 1/0

With favorable one as 1. 

keep all other features

Race Is not dummy columned, thus we have fewer columns


# Huangrui defined Adult Dataset

In [2]:
default_mappings = {
    'label_maps': [{1.0: '>50K', 0.0: '<=50K'}],
    'protected_attribute_maps': [{1.0: 'Non-Black', 0.0: 'Black'},
                                 {1.0: 'Male', 0.0: 'Female'}]
}
def code_continuous(df,collist,Nlevel):
    for col in collist:
        for q in range(1,Nlevel,1):
            threshold = df[~np.isnan(df[col])][col].quantile(float(q)/Nlevel)
            df[col+'_geq'+str(int(q))+'q'+str(threshold)] = (df[col] >= threshold).astype(float)
    df.drop(collist,axis = 1, inplace = True)
class AdultDataset(StandardDataset):
    """Adult Census Income Dataset.

    See :file:`aif360/data/raw/adult/README.md`.
    """

    def __init__(self, label_name='Y',
                 favorable_classes=[1],
                 protected_attribute_names=["race_Black"],
                 privileged_classes=[ [0]],
                 instance_weights_name=None,
                 categorical_features=[],
                 features_to_keep=[], features_to_drop=[],
                 na_values=['?'], custom_preprocessing=None,
                 metadata=default_mappings, path=None):
         #deal with the race
         df = pd.read_csv("./Huangrui/adult/adult_train1.csv",index_col=False,header = 0, sep = ',')
         numericals = [col for col in df.columns if len(df[col].unique())>2 and max(df[col])>1]
         code_continuous(df,numericals, 5)
         df.drop(["race_Asian-Pac-Islander",
            "race_Amer-Indian-Eskimo",
            "race_White",
            "race_Other"], axis=1, inplace=True)
         super(AdultDataset, self).__init__(df=df, label_name=label_name,
            favorable_classes=favorable_classes,
            protected_attribute_names=protected_attribute_names,
            privileged_classes=privileged_classes,
            instance_weights_name=instance_weights_name,
            categorical_features=categorical_features,
            features_to_keep=features_to_keep,
            features_to_drop=features_to_drop, na_values=na_values,
            custom_preprocessing=custom_preprocessing, metadata=metadata)

In [3]:
## import dataset
dataset_used = "adult" # "adult", "german", "compas"

privileged_groups = [{'race_Black': 0}]
unprivileged_groups = [{'race_Black': 1}]

# Metric used (should be one of allowed_metrics)
metric_name = "Equal opportunity difference"

# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
        
#random seed for calibrated equal odds prediction
random_seed = 12345679
np.random.seed(random_seed)

# Verify metric name
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
if metric_name not in allowed_metrics:
    raise ValueError("Metric name should be one of allowed metrics")

#### Split into train, test and validation

change the dataset_train[i] for specific K

In [40]:
K=5
budget = 0.01 #0.01, 0.1, 1

In [41]:
dataset_orig_train= AdultDataset(path="./Huangrui/adult/adult_train{}.csv".format(K))

In [42]:
dataset_orig_test= AdultDataset(path="./Huangrui/adult/adult_test{}.csv".format(K))

In [43]:
#only use the budget% of the training data
dataset_orig_train,_ = dataset_orig_train.split([budget], shuffle=False)

#### Clean up training data and display properties of the data

In [44]:
# print out some labels, names, etc.
display(Markdown("#### Training Dataset shape"))
print(dataset_orig_train.features.shape)
display(Markdown("#### Favorable and unfavorable labels"))
print(dataset_orig_train.favorable_label, dataset_orig_train.unfavorable_label)
display(Markdown("#### Protected attribute names"))
print(dataset_orig_train.protected_attribute_names)
display(Markdown("#### Privileged and unprivileged protected attribute values"))
print(dataset_orig_train.privileged_protected_attributes, 
      dataset_orig_train.unprivileged_protected_attributes)
display(Markdown("#### Dataset feature names"))
print(dataset_orig_train.feature_names)

#### Training Dataset shape

(239, 122)


#### Favorable and unfavorable labels

1.0 0.0


#### Protected attribute names

['race_Black']


#### Privileged and unprivileged protected attribute values

[array([0.])] [array([1.])]


#### Dataset feature names

['workclass_?', 'workclass_Federal-gov', 'workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc', 'workclass_State-gov', 'workclass_Without-pay', 'education_10th', 'education_11th', 'education_12th', 'education_1st-4th', 'education_5th-6th', 'education_7th-8th', 'education_9th', 'education_Assoc-acdm', 'education_Assoc-voc', 'education_Bachelors', 'education_Doctorate', 'education_HS-grad', 'education_Masters', 'education_Preschool', 'education_Prof-school', 'education_Some-college', 'maritalstatus_Divorced', 'maritalstatus_Married-AF-spouse', 'maritalstatus_Married-civ-spouse', 'maritalstatus_Married-spouse-absent', 'maritalstatus_Never-married', 'maritalstatus_Separated', 'maritalstatus_Widowed', 'occupation_?', 'occupation_Adm-clerical', 'occupation_Armed-Forces', 'occupation_Craft-repair', 'occupation_Exec-managerial', 'occupation_Farming-fishing', 'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct', 'o

#### Metric for original training data

In [45]:
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_orig_train.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = -0.037443


### Train classifier on original data

In [46]:
dataset_orig_train

               instance weights    features                        \
                                                                    
                                workclass_? workclass_Federal-gov   
instance names                                                      
0                           1.0         0.0                   0.0   
1                           1.0         0.0                   0.0   
2                           1.0         0.0                   0.0   
3                           1.0         0.0                   0.0   
4                           1.0         0.0                   0.0   
...                         ...         ...                   ...   
234                         1.0         0.0                   0.0   
235                         1.0         0.0                   0.0   
236                         1.0         0.0                   0.0   
237                         1.0         0.0                   0.0   
238                         1.0   

In [47]:
# Logistic regression classifier and predictions
scale_orig = StandardScaler()
X_train = scale_orig.fit_transform(dataset_orig_train.features)
y_train = dataset_orig_train.labels.ravel()

lmod = pickle.load(open('experiments/adult'+str(K)+'_race_bmodel.pkl','rb'))['clf']
y_train_pred = lmod.predict(X_train)

dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
dataset_orig_train_pred.labels = y_train_pred.reshape(-1,1)
sigmoid = lambda x: 1 / (1 + np.exp(0.5-x))
dataset_orig_train_pred.scores = sigmoid(y_train_pred).reshape(-1,1)

C:\edb\languagepack\v2\Python-3.9\lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Lasso from version 0.22.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


#### Obtain scores for test sets

In [48]:
dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
X_test = scale_orig.transform(dataset_orig_test_pred.features)
y_test = dataset_orig_test_pred.labels
dataset_orig_test_pred.scores = sigmoid(lmod.predict(X_test)).reshape(-1,1)

### Find the optimal parameters from the training set with specific budget

#### Best threshold for classification only (no fairness)

In [49]:
num_thresh = 100
ba_arr = np.zeros(num_thresh)
class_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
for idx, class_thresh in enumerate(class_thresh_arr):
    
    fav_inds = dataset_orig_train_pred.scores > class_thresh
    dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
    dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label
    
    classified_metric_orig_train = ClassificationMetric(dataset_orig_train,
                                             dataset_orig_train_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups=privileged_groups)
    
    ba_arr[idx] = 0.5*(classified_metric_orig_train.true_positive_rate()\
                       +classified_metric_orig_train.true_negative_rate())

best_ind = np.where(ba_arr == np.max(ba_arr))[0][0]
best_class_thresh = class_thresh_arr[best_ind]

print("Best balanced accuracy (no fairness constraints) = %.4f" % np.max(ba_arr))
print("Optimal classification threshold (no fairness constraints) = %.4f" % best_class_thresh)

Best balanced accuracy (no fairness constraints) = 0.8242
Optimal classification threshold (no fairness constraints) = 0.3565


#### Estimate optimal parameters for the ROC method

In [50]:
ROC = RejectOptionClassification(unprivileged_groups=unprivileged_groups, 
                                 privileged_groups=privileged_groups, 
                                 low_class_thresh=0.01, high_class_thresh=0.99,
                                  num_class_thresh=100, num_ROC_margin=50,
                                  metric_name=metric_name,
                                  metric_ub=metric_ub, metric_lb=metric_lb)
ROC = ROC.fit(dataset_orig_train, dataset_orig_train_pred)

In [51]:
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC.ROC_margin)

Optimal classification threshold (with fairness constraints) = 0.2971
Optimal ROC margin = 0.0121


### Predictions from train Set

In [52]:
# Metrics for the test set
fav_inds = dataset_orig_train_pred.scores > best_class_thresh
dataset_orig_train_pred.labels[fav_inds] = dataset_orig_train_pred.favorable_label
dataset_orig_train_pred.labels[~fav_inds] = dataset_orig_train_pred.unfavorable_label

display(Markdown("#### train set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_train_bef = compute_metrics(dataset_orig_train, dataset_orig_train_pred, 
                unprivileged_groups, privileged_groups)

#### traination set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.8242
Statistical parity difference = -0.1247
Disparate impact = 0.7065
Average odds difference = -0.0271
Equal opportunity difference = 0.0962
Theil index = 0.0705


In [53]:
# Transform the validation set
dataset_transf_train_pred = ROC.predict(dataset_orig_train_pred)

display(Markdown("#### train set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_train_aft = compute_metrics(dataset_orig_train, dataset_transf_train_pred, 
                unprivileged_groups, privileged_groups)

#### train set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.7499
Statistical parity difference = -0.0890
Disparate impact = 0.8488
Average odds difference = -0.0298
Equal opportunity difference = 0.0385
Theil index = 0.0679


In [54]:
# Testing: Check if the metric optimized has not become worse
assert np.abs(metric_train_aft[metric_name]) <= np.abs(metric_train_bef[metric_name])

### Predictions from Test Set

In [55]:
# Metrics for the test set
fav_inds = dataset_orig_test_pred.scores > best_class_thresh
dataset_orig_test_pred.labels[fav_inds] = dataset_orig_test_pred.favorable_label
dataset_orig_test_pred.labels[~fav_inds] = dataset_orig_test_pred.unfavorable_label

display(Markdown("#### Test set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))

metric_test_bef = compute_metrics(dataset_orig_test, dataset_orig_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.7971
Statistical parity difference = -0.1884
Disparate impact = 0.5545
Average odds difference = -0.0906
Equal opportunity difference = -0.0601
Theil index = 0.0844


In [56]:
# Metrics for the transformed test set
dataset_transf_test_pred = ROC.predict(dataset_orig_test_pred)

display(Markdown("#### Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_test_aft = compute_metrics(dataset_orig_test, dataset_transf_test_pred, 
                unprivileged_groups, privileged_groups)

#### Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.7628
Statistical parity difference = -0.1446
Disparate impact = 0.7395
Average odds difference = -0.0425
Equal opportunity difference = 0.0054
Theil index = 0.0726


In [57]:
#自己计算error, 不是balanced accuracy！！！
metric_test_aft["Equal opportunity difference"]
print("The Error for the test dataset is {:.4}".format(np.mean(dataset_orig_test.labels!=dataset_transf_test_pred.labels)))
print("The Equal opportunity difference for the test dataset is {:.4}".format(metric_test_aft["Equal opportunity difference"]))

The Error for the test dataset is 0.3303
The Equal opportunity difference for the test dataset is 0.005392
